In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
model = open(r'C:\Users\KIIT\Documents\MachineLearning_scripts\Flight Fare prediction/tuned_rf_model.pkl','rb')
forest = pickle.load(model)

In [3]:
test_df = pd.read_excel(r"C:\Users\KIIT\Documents\MachineLearning_scripts\Flight Fare prediction/Test_set.xlsx")
test_df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info
...,...,...,...,...,...,...,...,...,...,...
2666,Air India,6/06/2019,Kolkata,Banglore,CCU → DEL → BLR,20:30,20:25 07 Jun,23h 55m,1 stop,No info
2667,IndiGo,27/03/2019,Kolkata,Banglore,CCU → BLR,14:20,16:55,2h 35m,non-stop,No info
2668,Jet Airways,6/03/2019,Delhi,Cochin,DEL → BOM → COK,21:50,04:25 07 Mar,6h 35m,1 stop,No info
2669,Air India,6/03/2019,Delhi,Cochin,DEL → BOM → COK,04:00,19:15,15h 15m,1 stop,No info


In [4]:
test_df.dropna(inplace=True)

In [5]:
def change_into_datetime(col):
    test_df[col] = pd.to_datetime(test_df[col])

In [6]:
for i in ['Date_of_Journey','Dep_Time','Arrival_Time']:
    change_into_datetime(i)

In [7]:
test_df['Journey_date'] = test_df['Date_of_Journey'].dt.day
test_df['Journey_month'] = test_df['Date_of_Journey'].dt.month
test_df['Journey_year'] = test_df['Date_of_Journey'].dt.year

In [8]:
def extract_hour(test_df,col):
    test_df[col + '_hour'] = test_df[col].dt.hour

def extract_minute(train_df,col):
    test_df[col + '_minute'] = test_df[col].dt.minute

In [9]:
extract_hour(test_df,'Dep_Time')
extract_minute(test_df,'Dep_Time')

extract_hour(test_df,'Arrival_Time')
extract_minute(test_df,'Arrival_Time')

In [10]:
test_df.drop(columns=['Arrival_Time','Dep_Time'], axis=1, inplace=True)

In [11]:
duration = list(test_df['Duration'])
for i in range (len(duration)):
    if len(duration[i].split(' '))==2:
        pass
    else:
        if 'h' in duration[i]:
              duration[i] = duration[i] + ' 0m'
        else:
              duration[i] = '0h ' + duration[i]

In [12]:
test_df['Duration'] = duration

In [13]:
def hour(x):
    return x.split(' ')[0][0:-1]

def minute(x):
    return x.split(' ')[1][0:-1]

In [14]:
test_df['Duration Hours'] = test_df['Duration'].apply(hour).astype(int)
test_df['Duration Minutes'] = test_df['Duration'].apply(minute).astype(int)

In [15]:
test_df.drop(['Duration'],axis=1,inplace=True)

In [16]:
categorical_col = [col for col in test_df.columns if test_df[col].dtype=='O']
categorical_col

numerical_col = [col for col in test_df.columns if test_df[col].dtype!='O']
numerical_col

['Date_of_Journey',
 'Journey_date',
 'Journey_month',
 'Journey_year',
 'Dep_Time_hour',
 'Dep_Time_minute',
 'Arrival_Time_hour',
 'Arrival_Time_minute',
 'Duration Hours',
 'Duration Minutes']

In [17]:
categorical = test_df[categorical_col]

In [18]:
Airlines_df = pd.get_dummies(categorical['Airline'], drop_first=True)

Destination_df = pd.get_dummies(categorical['Destination'], drop_first=True)

Source_df = pd.get_dummies(categorical['Source'], drop_first=True)

route_lst = categorical['Route'].str.split('→')
categorical['Route_1'] = route_lst.str[0]
categorical['Route_2'] = route_lst.str[1]
categorical['Route_3'] = route_lst.str[2]
categorical['Route_4'] = route_lst.str[3]
categorical['Route_5'] = route_lst.str[4]


C:\Users\KIIT\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\KIIT\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\KIIT\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [19]:
for i in ['Route_3','Route_4','Route_5']:
    categorical[i].fillna('None',inplace=True)

C:\Users\KIIT\anaconda3\lib\site-packages\pandas\core\series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [21]:
for i in ['Route_1','Route_2','Route_3','Route_4','Route_5']:
    categorical[i] = le.fit_transform(categorical[i])

C:\Users\KIIT\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
categorical.drop(['Route','Additional_Info'],axis=1,inplace=True)

C:\Users\KIIT\anaconda3\lib\site-packages\pandas\core\frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
dict = {'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, '4 stops':4}

categorical['Total_Stops'] = categorical['Total_Stops'].map(dict)

C:\Users\KIIT\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
test_data = pd.concat([categorical,Airlines_df,Source_df,Destination_df,test_df[numerical_col]],axis=1)

In [25]:
test_data.drop(['Airline','Source','Destination','Date_of_Journey'],axis=1,inplace=True)

In [26]:
test_data.head()

,Total_Stops,Route_1,Route_2,Route_3,Route_4,Route_5,Air India,GoAir,IndiGo,Jet Airways,...,New Delhi,Journey_date,Journey_month,Journey_year,Dep_Time_hour,Dep_Time_minute,Arrival_Time_hour,Arrival_Time_minute,Duration Hours,Duration Minutes
0,1,3,7,5,9,4,0,0,0,1,...,0,6,6,2019,17,30,4,25,10,55
1,1,2,29,3,9,4,0,0,1,0,...,0,5,12,2019,6,20,10,20,4,0
2,1,3,7,5,9,4,0,0,0,1,...,0,21,5,2019,19,15,19,0,23,45
3,1,3,7,5,9,4,0,0,0,0,...,0,21,5,2019,8,0,21,0,13,0
4,0,0,13,27,9,4,0,0,0,0,...,0,24,6,2019,23,55,2,45,2,50


In [27]:
test_data.shape

(2671, 34)

In [28]:
test_preds = forest.predict(test_data)

In [29]:
test_preds

array([11662.8295996 ,  5654.82204972, 12409.77167058, ...,
       12737.48748217,  9947.78307414,  9261.54009843])

In [30]:
test_df['Predicted Price'] = test_preds

In [31]:
test_df

,Airline,Date_of_Journey,Source,Destination,Route,Total_Stops,Additional_Info,Journey_date,Journey_month,Journey_year,Dep_Time_hour,Dep_Time_minute,Arrival_Time_hour,Arrival_Time_minute,Duration Hours,Duration Minutes,Predicted Price
0,Jet Airways,2019-06-06,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,6,6,2019,17,30,4,25,10,55,11662.829600
1,IndiGo,2019-12-05,Kolkata,Banglore,CCU → MAA → BLR,1 stop,No info,5,12,2019,6,20,10,20,4,0,5654.822050
2,Jet Airways,2019-05-21,Delhi,Cochin,DEL → BOM → COK,1 stop,In-flight meal not included,21,5,2019,19,15,19,0,23,45,12409.771671
3,Multiple carriers,2019-05-21,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,21,5,2019,8,0,21,0,13,0,11780.628704
4,Air Asia,2019-06-24,Banglore,Delhi,BLR → DEL,non-stop,No info,24,6,2019,23,55,2,45,2,50,6362.034529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,Air India,2019-06-06,Kolkata,Banglore,CCU → DEL → BLR,1 stop,No info,6,6,2019,20,30,20,25,23,55,11330.447939
2667,IndiGo,2019-03-27,Kolkata,Banglore,CCU → BLR,non-stop,No info,27,3,2019,14,20,16,55,2,35,10195.648021
2668,Jet Airways,2019-06-03,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,3,6,2019,21,50,4,25,6,35,12737.487482
2669,Air India,2019-06-03,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,3,6,2019,4,0,19,15,15,15,9947.783074


In [32]:
test_df.to_excel(r'C:\Users\KIIT\Documents\MachineLearning_scripts\Flight Fare prediction\Test_Predictions.xlsx')